Libraries

In [ ]:
import operator
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import snntorch as snn
import torch    
import torchvision
from deap import base, creator, tools
from rich.console import Console
from rich.traceback import install
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import pandas as pd
from matplotlib.patches import Rectangle

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

MNIST data set loader

In [ ]:
install(show_locals=True)
console = Console(width=180)
RNG = np.random.default_rng()
plt.style.use("seaborn-v0_8")
sns.set_palette("husl")
backprop_df = pd.read_csv("src/notebooks/accuracy_log.csv")
backprop_accuracies = backprop_df["accuracy"].tolist()
DEVICE = "cpu"
if torch.cuda.is_available():
    DEVICE = "cuda"
elif torch.backends.mps.is_available():
    DEVICE = "mps"
POP_SIZE = 12

transform = transforms.Compose([transforms.ToTensor()])
train_data = torchvision.datasets.MNIST(
    root="./data",
    train=True,
    download=True,
    transform=transform,
)
test_data = torchvision.datasets.MNIST(
    root="./data",
    train=False,
    download=True,
    transform=transform,
)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

creating SNN

In [ ]:
def create_snn(layer_sizes, beta, dropout_rate=0.1):

    class SNNModel(nn.Module):
        def __init__(self):
            super().__init__()

            self.layers = nn.ModuleList()
            self.lif_layers = nn.ModuleList()
            self.dropout_layers = nn.ModuleList()

            input_size = 28 * 28
            num_of_classes = 10

            for i, hidden_size in enumerate(layer_sizes):
                self.layers.append(nn.Linear(input_size, hidden_size))
                self.lif_layers.append(snn.Leaky(beta=beta))

                if i < len(layer_sizes) - 1:
                    self.dropout_layers.append(nn.Dropout(dropout_rate))
                else:
                    self.dropout_layers.append(nn.Identity())

                input_size = hidden_size

            self.layers.append(nn.Linear(input_size, num_of_classes))

            self.apply(self._init_weights)

        def _init_weights(self, m):
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight, gain=0.8)
                if m.bias is not None:
                    m.bias.data.fill_(0.01)

        def forward(self, x, num_steps=15):  
            mem_states = [lif.init_leaky() for lif in self.lif_layers]
            spk_out = 0

            for step in range(num_steps):
                current_input = x.view(x.size(0), -1)

                for i, (layer, lif, dropout) in enumerate(
                    zip(
                        self.layers,
                        self.lif_layers,
                        self.dropout_layers + [nn.Identity()],
                    )
                ):
                    current = layer(current_input)
                    spike, mem_states[i] = lif(current, mem_states[i])
                    if (
                        i < len(self.layers) - 1
                    ):  
                        spike = dropout(spike)
                    current_input = spike

                spk_out += current_input

            return spk_out / num_steps

    return SNNModel()


Creating mutation

In [ ]:
def create_individual_with_depth():
    """Create individual with improved parameter ranges and depth diversity."""
    # Bias toward mid-range depths (3-6 layers) but allow full range
    depth_weights = [0.05, 0.05, 0.05, 0.20, 0.25, 0.20, 0.10, 0.05, 0.05]  # Favor 4-6 layers more strongly
    num_hidden_layers = np.random.choice(range(1, 10), p=depth_weights) 

    # Create layer sizes with tapering (wider to narrower)
    layer_sizes = []
    base_size = 512

    for i in range(num_hidden_layers):
        # Gradually reduce size for deeper layers
        reduction_factor = 0.85**i
        size = max(64, int(base_size * reduction_factor + RNG.uniform(-30, 30)))
        layer_sizes.append(size)

    # Depth-adaptive parameters
    beta = RNG.uniform(0.6, 0.90)  # Wider beta range
    
    # Adjust learning rate based on depth - deeper networks need lower LR
    base_lr = RNG.uniform(0.0008, 0.005)
    lr = base_lr / (1 + num_hidden_layers * 0.03)  # Lower LR for deeper networks
    
    dropout = RNG.uniform(0.05, 0.3)  # Slightly higher dropout range

    # [num_hidden_layers, *layer_sizes, beta, lr, dropout]
    return [num_hidden_layers, *layer_sizes, beta, lr, dropout]


def bounded_mutation_with_depth(individual, mu=0, sigma=0.15, indpb=0.2):
    """Improved mutation with better depth exploration."""
    num_layers = int(individual[0])

    # Mutate number of layers (increased probability)
    if RNG.random() < 0.7:  # Increased from 0.15
        # Allow both adding and removing layers
        change = 1 if RNG.random() < 0.75 else -1 # Can remove, stay same, or add
        new_num_layers = max(2, min(8, num_layers + change))

        if new_num_layers > num_layers:
            # Add a new layer (smaller than previous)
            prev_size = individual[num_layers] if num_layers > 0 else 128
            new_layer_size = max(32, int(prev_size * 0.7 + RNG.uniform(-10, 10)))
            individual.insert(num_layers + 1, new_layer_size)
        elif new_num_layers < num_layers:
            # Remove a layer (remove from middle preferentially)
            if num_layers > 1:
                remove_idx = RNG.choice(range(1, min(num_layers + 1, len(individual) - 3)))
                individual.pop(remove_idx)

        individual[0] = new_num_layers
        num_layers = new_num_layers

    # Mutate layer sizes with smaller perturbations
    for i in range(1, num_layers + 1):
        if i < len(individual) - 3 and RNG.random() < indpb:
            individual[i] += RNG.normal(mu, sigma * 30)  # Smaller mutations
            individual[i] = max(32, min(512, individual[i]))  # Wider range

    # Mutate beta (wider range)
    if len(individual) > num_layers + 1 and RNG.random() < indpb:
        individual[num_layers + 1] += RNG.normal(mu, sigma * 0.08)
        individual[num_layers + 1] = max(0.5, min(0.9, individual[num_layers + 1]))

    # Mutate learning rate
    if len(individual) > num_layers + 2 and RNG.random() < indpb:
        individual[num_layers + 2] += RNG.normal(mu, sigma * 0.002)
        individual[num_layers + 2] = max(0.0005, min(0.01, individual[num_layers + 2]))

    # Mutate dropout
    if len(individual) > num_layers + 3 and RNG.random() < indpb:
        individual[num_layers + 3] += RNG.normal(mu, sigma * 0.05)
        individual[num_layers + 3] = max(0.0, min(0.3, individual[num_layers + 3]))

    return (individual,)


def crossover_with_depth(ind1, ind2):
    """Improved crossover for variable-length individuals with diversity preservation."""
    num_layers1 = int(ind1[0])
    num_layers2 = int(ind2[0])

    # Sometimes choose depth randomly to maintain diversity
    if RNG.random() < 0.3:  # 30% chance for random depth choice
        chosen_depth = RNG.choice([num_layers1, num_layers2])
    else:
        # Choose depth more intelligently (favor successful depths)
        if hasattr(ind1, "fitness") and hasattr(ind2, "fitness"):
            if ind1.fitness.valid and ind2.fitness.valid:
                if ind1.fitness.values[0] > ind2.fitness.values[0]:
                    chosen_depth = num_layers1
                else:
                    chosen_depth = num_layers2
            else:
                chosen_depth = RNG.choice([num_layers1, num_layers2])
        else:
            chosen_depth = RNG.choice([num_layers1, num_layers2])

    # Create new individuals
    new_ind1 = [chosen_depth]
    new_ind2 = [chosen_depth]

    # Blend layer sizes
    for i in range(chosen_depth):
        if i < num_layers1 and i < num_layers2:
            # Blend existing layers
            alpha = RNG.uniform(0.3, 0.7)  # Less extreme blending
            size1 = alpha * ind1[1 + i] + (1 - alpha) * ind2[1 + i]
            size2 = (1 - alpha) * ind1[1 + i] + alpha * ind2[1 + i]
        elif i < num_layers1:
            size1 = ind1[1 + i] + RNG.uniform(-10, 10)
            size2 = RNG.uniform(32, 256)
        elif i < num_layers2:
            size1 = RNG.uniform(32, 256)
            size2 = ind2[1 + i] + RNG.uniform(-10, 10)
        else:
            size1 = RNG.uniform(64, 256)
            size2 = RNG.uniform(64, 256)

        new_ind1.append(max(32, min(512, size1)))
        new_ind2.append(max(32, min(512, size2)))

    # Blend other parameters
    alpha = RNG.uniform(0.3, 0.7)
    beta1 = alpha * ind1[num_layers1 + 1] + (1 - alpha) * ind2[num_layers2 + 1]
    beta2 = (1 - alpha) * ind1[num_layers1 + 1] + alpha * ind2[num_layers2 + 1]
    lr1 = alpha * ind1[num_layers1 + 2] + (1 - alpha) * ind2[num_layers2 + 2]
    lr2 = (1 - alpha) * ind1[num_layers1 + 2] + alpha * ind2[num_layers2 + 2]
    dropout1 = (
        alpha * ind1[num_layers1 + 3] + (1 - alpha) * ind2[num_layers2 + 3]
    )
    dropout2 = (1 - alpha) * ind1[num_layers1 + 3] + alpha * ind2[
        num_layers2 + 3
    ]

    new_ind1.extend([beta1, lr1, dropout1])
    new_ind2.extend([beta2, lr2, dropout2])

    ind1[:] = new_ind1
    ind2[:] = new_ind2

    return ind1, ind2


def evaluate_model_with_depth(individual):
    """Improved evaluation with depth-adaptive training and better debugging."""
    try:
        # Parse individual
        num_layers = int(individual[0])
        layer_sizes = [
            max(64, min(768, round(individual[i])))
            for i in range(1, num_layers + 1)
        ]
        beta = max(0.5, min(0.95, float(individual[num_layers + 1])))
        lr = max(0.0003, min(0.008, float(individual[num_layers + 2])))
        dropout = max(0.0, min(0.4, float(individual[num_layers + 3])))

        # Debug: Print individual parameters
        console.print(f"Training: Layers={layer_sizes}, Beta={beta:.3f}, LR={lr:.4f}, Dropout={dropout:.3f}")

        model = create_snn(layer_sizes, beta, dropout)
        model.to(DEVICE)
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=lr,
            weight_decay=5e-4,
            betas=(0.9, 0.999)
        )
        loss_fn = nn.CrossEntropyLoss()

        # Depth-adaptive training - deeper networks get more training
        max_batches = min(1200, 400 + num_layers * 100)
        
        # Extended training loop
        model.train()
        total_loss = 0
        num_batches = 0

        for batch_idx, (data, targets) in enumerate(train_loader):
            data, targets = data.to(DEVICE), targets.to(DEVICE)
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, targets)
            loss.backward()

            # Depth-adaptive gradient clipping
            if num_layers > 4:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
            elif num_layers > 2:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.8)
            else:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()
            total_loss += loss.item()
            num_batches += 1

            if batch_idx >= max_batches:
                break

        # More comprehensive evaluation
        model.eval()
        correct, total = 0, 0
        eval_loss = 0

        with torch.no_grad():
            for batch_idx, (data, targets) in enumerate(test_loader):
                data, targets = data.to(DEVICE), targets.to(DEVICE)
                outputs = model(data)
                loss = loss_fn(outputs, targets)
                eval_loss += loss.item()

                _, pred = outputs.max(1)
                total += targets.size(0)
                correct += (pred == targets).sum().item()

                # Evaluate on more samples for better accuracy
                if total >= 3000:  # Increased from 2000
                    break

        accuracy = correct / total
        avg_train_loss = total_loss / num_batches
        avg_eval_loss = eval_loss / (batch_idx + 1)
        #commit
        # Debug: Print training details
        console.print(f"Training Loss: {avg_train_loss:.4f}, Eval Loss: {avg_eval_loss:.4f}")
        console.print(f"Correct: {correct}/{total} = {accuracy:.4f}")

        # Small complexity penalty to encourage efficient architectures
        complexity_penalty = (num_layers * 0.001 + sum(layer_sizes) * 0.000005)
        final_fitness = accuracy

        console.print(f"Final Fitness: {final_fitness:.6f} (Accuracy: {accuracy:.4f}, Penalty: {complexity_penalty:.6f})")
        return (final_fitness,)

    except Exception as e:
        console.print(f"Evaluation ERROR: {str(e)}")

        return (-1.0,)

Creating plotting

In [ ]:
def count_parameters(layer_sizes: list[int]) -> int:
    # Full layer structure: input (784) + hidden layers + output (10)
    full_layers = [784] + layer_sizes + [10]
    total = 0
    for i in range(len(full_layers) - 1):
        input_size = full_layers[i]
        output_size = full_layers[i + 1]
        total += (input_size * output_size) + output_size  # weights + biases
    return total

def parse_individual(individual):
    """Helper function to parse individual parameters."""
    num_layers = int(individual[0])
    layer_sizes = [round(individual[i]) for i in range(1, num_layers + 1)]
    beta = individual[num_layers + 1]
    lr = individual[num_layers + 2]
    dropout = individual[num_layers + 3]
    return num_layers, layer_sizes, beta, lr, dropout


def print_best_individual(generation, best_individual, fitness) -> None:
    """Print detailed information about the best individual."""
    _num_layers, layer_sizes, beta, lr, dropout = parse_individual(
        best_individual,
    )

    console.print(f"\n{'=' * 60}")
    console.print(f"GENERATION {generation} - BEST INDIVIDUAL")
    console.print(f"{'=' * 60}")
    console.print(f"Fitness: {fitness:.6f}")
    console.print(
        f"Architecture: 784 → {' → '.join(map(str, layer_sizes))} → 10",
    )
    console.print(f"Beta (β): {beta:.4f}")
    console.print(f"Learning Rate: {lr:.6f}")
    console.print(f"Dropout Rate: {dropout:.4f}")
    console.print(
    f"Total Parameters: ~{count_parameters(layer_sizes):,}",
)
    console.print(f"{'=' * 60}")


def plot_evolution_progress(logbook, generation_best) -> None:
    """Plot evolution progress with multiple metrics."""
    _fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

    # Extract data
    generations = [record["gen"] for record in logbook]
    max_fitness = [record["max"] for record in logbook]
    avg_fitness = [record["avg"] for record in logbook]
    min_fitness = [record["min"] for record in logbook]
    std_fitness = [record["std"] for record in logbook]

    # Plot 1: Fitness Evolution
    ax1.plot(
        generations,
        max_fitness,
        "r-",
        linewidth=2,
        label="Max Fitness",
        marker="o",
    )
    ax1.plot(
        generations,
        avg_fitness,
        "b-",
        linewidth=2,
        label="Avg Fitness",
        marker="s",
    )
    ax1.fill_between(
        generations,
        [avg - std for avg, std in zip(avg_fitness, std_fitness, strict=False)],
        [avg + std for avg, std in zip(avg_fitness, std_fitness, strict=False)],
        alpha=0.3,
        color="blue",
    )
    ax1.set_xlabel("Generation")
    ax1.set_ylabel("Fitness")
    ax1.set_title("Evolution Progress - Fitness Over Time")
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Plot 2: Best Fitness Trajectory
    best_gens = [item[0] for item in generation_best]
    best_fits = [item[2] for item in generation_best]
    ax2.plot(best_gens, best_fits, "g-", linewidth=3, marker="*", markersize=8)
    ax2.set_xlabel("Generation")
    ax2.set_ylabel("Best Fitness")
    ax2.set_title("Best Individual Fitness Trajectory")
    ax2.grid(True, alpha=0.3)

    # Annotate improvements
    for i in range(1, len(best_fits)):
        if best_fits[i] > best_fits[i - 1]:
            ax2.annotate(
                f"↑ {best_fits[i]:.4f}",
                xy=(best_gens[i], best_fits[i]),
                xytext=(10, 10),
                textcoords="offset points",
                bbox={
                    "boxstyle": "round,pad=0.3",
                    "facecolor": "yellow",
                    "alpha": 0.7,
                },
                arrowprops={
                    "arrowstyle": "->",
                    "connectionstyle": "arc3,rad=0",
                },
            )

    # Plot 3: Fitness Distribution
    try:
        final_gen_data = [
            ind.fitness.values[0]
            for ind in logbook[-1]["population"]
            if hasattr(ind, "fitness") and ind.fitness.valid
        ]
    except KeyError:
        final_gen_data = [max_fitness[-1], avg_fitness[-1], min_fitness[-1]]

    ax3.hist(
        final_gen_data,
        bins=15,
        alpha=0.7,
        color="purple",
        edgecolor="black",
    )
    ax3.axvline(
        np.mean(final_gen_data),
        color="red",
        linestyle="--",
        label=f"Mean: {np.mean(final_gen_data):.4f}",
    )
    ax3.set_xlabel("Fitness")
    ax3.set_ylabel("Frequency")
    ax3.set_title("Final Generation Fitness Distribution")
    ax3.legend()
    ax3.grid(True, alpha=0.3)

    # Plot 4: Convergence Analysis
    improvement_rate = []
    for i in range(1, len(max_fitness)):
        rate = max_fitness[i] - max_fitness[i - 1]
        improvement_rate.append(rate)

    ax4.plot(
        generations[1:],
        improvement_rate,
        "orange",
        linewidth=2,
        marker="d",
    )
    ax4.axhline(y=0, color="black", linestyle="-", alpha=0.3)
    ax4.set_xlabel("Generation")
    ax4.set_ylabel("Fitness Improvement")
    ax4.set_title("Convergence Rate (Generation-to-Generation Improvement)")
    ax4.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig("plots/evolution_progress.png", dpi=300, bbox_inches="tight")
    plt.show()


def plot_architecture_evolution(generation_best) -> None:
    """Plot architecture evolution over generations."""
    _fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

    # Extract architecture data
    generations = []
    num_layers_list = []
    layer_sizes_by_gen = []
    total_params = []

    for gen, individual, _fitness in generation_best:
        generations.append(gen)
        num_layers, layer_sizes, _beta, _lr, _dropout = parse_individual(
            individual,
        )
        num_layers_list.append(num_layers)
        layer_sizes_by_gen.append(layer_sizes)

        # Calculate approximate parameter count
        params = 784 * layer_sizes[0]  # Input to first hidden
        for i in range(len(layer_sizes) - 1):
            params += layer_sizes[i] * layer_sizes[i + 1]
        params += layer_sizes[-1] * 10  # Last hidden to output
        total_params.append(params)

    # Plot 1: Number of Layers Evolution
    ax1.plot(generations, num_layers_list, "b-o", linewidth=2, markersize=8)
    ax1.set_xlabel("Generation")
    ax1.set_ylabel("Number of Hidden Layers")
    ax1.set_title("Architecture Depth Evolution")
    ax1.set_ylim(0.5, 10.5)
    ax1.grid(True, alpha=0.3)

    # Annotate layer changes
    for i in range(1, len(num_layers_list)):
        if num_layers_list[i] != num_layers_list[i - 1]:
            change = "+" if num_layers_list[i] > num_layers_list[i - 1] else "-"
            ax1.annotate(
                f"{change}Layer",
                xy=(generations[i], num_layers_list[i]),
                xytext=(10, 10),
                textcoords="offset points",
                bbox={"boxstyle": "round,pad=0.3", "facecolor": "lightblue"},
                arrowprops={"arrowstyle": "->"},
            )

    # Plot 2: Layer Sizes Heatmap
    max_layers = max(len(sizes) for sizes in layer_sizes_by_gen)
    heatmap_data = np.zeros((len(generations), max_layers))

    for i, sizes in enumerate(layer_sizes_by_gen):
        for j, size in enumerate(sizes):
            heatmap_data[i, j] = size

    im = ax2.imshow(
        heatmap_data.T,
        cmap="viridis",
        aspect="auto",
        interpolation="nearest",
    )
    ax2.set_xlabel("Generation")
    ax2.set_ylabel("Layer Index")
    ax2.set_title("Layer Sizes Evolution Heatmap")
    ax2.set_xticks(range(0, len(generations), max(1, len(generations) // 10)))
    ax2.set_xticklabels(generations[:: max(1, len(generations) // 10)])
    plt.colorbar(im, ax=ax2, label="Layer Size")

    # Plot 3: Total Parameters Evolution
    ax3.plot(generations, total_params, "r-s", linewidth=2, markersize=6)
    ax3.set_xlabel("Generation")
    ax3.set_ylabel("Total Parameters")
    ax3.set_title("Model Complexity Evolution")
    ax3.grid(True, alpha=0.3)

    # Format y-axis to show K/M notation
    ax3.yaxis.set_major_formatter(
        plt.FuncFormatter(
            lambda x, p: f"{x / 1000:.0f}K"
            if x < 1000000
            else f"{x / 1000000:.1f}M",
        ),
    )

    # Plot 4: Architecture Visualization for Best Individual
    best_individual = max(generation_best, key=operator.itemgetter(2))
    _, best_layer_sizes, _, _, _ = parse_individual(best_individual[1])

    # Create network diagram
    ax4.set_xlim(0, 10)
    ax4.set_ylim(0, 10)

    # Draw layers
    layer_positions = np.linspace(
        1,
        9,
        len(best_layer_sizes) + 2,
    )  # +2 for input and output
    all_sizes = [784, *best_layer_sizes, 10]

    max_neurons = max(all_sizes)
    colors = plt.cm.viridis(np.linspace(0, 1, len(all_sizes)))

    for i, (pos, size, color) in enumerate(
        zip(layer_positions, all_sizes, colors, strict=False),
    ):
        # Scale rectangle height based on layer size
        height = (size / max_neurons) * 6 + 1
        y_center = 5

        rect = Rectangle(
            (pos - 0.3, y_center - height / 2),
            0.6,
            height,
            facecolor=color,
            edgecolor="black",
            alpha=0.7,
        )
        ax4.add_patch(rect)

        # Add labels
        if i == 0:
            label = f"Input\n{size}"
        elif i == len(all_sizes) - 1:
            label = f"Output\n{size}"
        else:
            label = f"H{i}\n{size}"

        ax4.text(
            pos,
            y_center - height / 2 - 0.5,
            label,
            ha="center",
            va="top",
            fontsize=8,
        )

        # Draw connections
        if i < len(layer_positions) - 1:
            ax4.arrow(
                pos + 0.3,
                y_center,
                0.4,
                0,
                head_width=0.1,
                head_length=0.1,
                fc="gray",
                ec="gray",
                alpha=0.5,
            )

    ax4.set_title(
        f"Best Architecture (Gen {best_individual[0]}, Fitness: {best_individual[2]:.4f})",
    )
    ax4.set_xticks([])
    ax4.set_yticks([])
    ax4.spines["top"].set_visible(False)
    ax4.spines["right"].set_visible(False)
    ax4.spines["bottom"].set_visible(False)
    ax4.spines["left"].set_visible(False)

    plt.tight_layout()
    plt.savefig(
        "plots/architecture_evolution.png",
        dpi=300,
        bbox_inches="tight",
    )
    plt.show()


def plot_beta_evolution(generation_best) -> None:
    """Plot beta parameter evolution and analysis."""
    _fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

    # Extract parameter data
    generations = []
    betas = []
    learning_rates = []
    dropouts = []
    fitnesses = []

    for gen, individual, fitness in generation_best:
        generations.append(gen)
        _num_layers, _layer_sizes, beta, lr, dropout = parse_individual(
            individual,
        )
        betas.append(beta)
        learning_rates.append(lr)
        dropouts.append(dropout)
        fitnesses.append(fitness)

    # Plot 1: Beta Evolution
    ax1.plot(
        generations,
        betas,
        "purple",
        linewidth=3,
        marker="o",
        markersize=8,
    )
    ax1.set_xlabel("Generation")
    ax1.set_ylabel("Beta Value")
    ax1.set_title("Beta Parameter Evolution")
    ax1.set_ylim(0.65, 1.0)
    ax1.grid(True, alpha=0.3)

    # Highlight significant changes
    for i in range(1, len(betas)):
        if abs(betas[i] - betas[i - 1]) > 0.05:
            ax1.annotate(
                f"β={betas[i]:.3f}",
                xy=(generations[i], betas[i]),
                xytext=(10, 10),
                textcoords="offset points",
                bbox={
                    "boxstyle": "round,pad=0.3",
                    "facecolor": "yellow",
                    "alpha": 0.7,
                },
                arrowprops={"arrowstyle": "->"},
            )

    # Plot 2: Learning Rate Evolution
    ax2.plot(
        generations,
        learning_rates,
        "green",
        linewidth=3,
        marker="s",
        markersize=6,
    )
    ax2.set_xlabel("Generation")
    ax2.set_ylabel("Learning Rate")
    ax2.set_title("Learning Rate Evolution")
    ax2.grid(True, alpha=0.3)
    ax2.ticklabel_format(style="scientific", axis="y", scilimits=(0, 0))

    # Plot 3: Parameter Correlation with Fitness
    # Create scatter plot with color coding for generations
    scatter = ax3.scatter(
        betas,
        fitnesses,
        c=generations,
        cmap="viridis",
        s=100,
        alpha=0.7,
        edgecolors="black",
    )
    ax3.set_xlabel("Beta Value")
    ax3.set_ylabel("Fitness")
    ax3.set_title("Beta vs Fitness Correlation")
    ax3.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax3, label="Generation")

    # Add trend line
    z = np.polyfit(betas, fitnesses, 1)
    p = np.poly1d(z)
    ax3.plot(
        betas,
        p(betas),
        "r--",
        alpha=0.8,
        linewidth=2,
        label=f"Trend: y={z[0]:.2f}x+{z[1]:.2f}",
    )
    ax3.legend()

    # Plot 4: All Parameters Over Time
    # Normalize parameters to [0, 1] for comparison
    epsilon = 1e-10  # Tiny value to prevent division by zero
    norm_betas = [
    (b - min(betas)) / (max(betas) - min(betas) + epsilon) 
    for b in betas
    ]
    norm_lrs = [
        (lr - min(learning_rates)) / (max(learning_rates) - min(learning_rates))
        for lr in learning_rates
    ]
    norm_dropouts = [
        (d - min(dropouts)) / (max(dropouts) - min(dropouts))
        if max(dropouts) > min(dropouts)
        else 0.5
        for d in dropouts
    ]

    ax4.plot(
        generations,
        norm_betas,
        "purple",
        linewidth=2,
        marker="o",
        label="Beta (normalized)",
    )
    ax4.plot(
        generations,
        norm_lrs,
        "green",
        linewidth=2,
        marker="s",
        label="Learning Rate (normalized)",
    )
    ax4.plot(
        generations,
        norm_dropouts,
        "orange",
        linewidth=2,
        marker="^",
        label="Dropout (normalized)",
    )

    ax4.set_xlabel("Generation")
    ax4.set_ylabel("Normalized Parameter Value")
    ax4.set_title("All Parameters Evolution (Normalized)")
    ax4.legend()
    ax4.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig("plots/beta_evolution.png", dpi=300, bbox_inches="tight")
    plt.show()


def plot_layers_evolution(logbook, generation_best) -> None:
    """Plot the evolution of layer counts in the population."""
    _fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # Extract data
    generations = [record["gen"] for record in logbook]
    avg_layers = [record["avg_layers"] for record in logbook]

    # Best individual layers
    best_layers = [ind[1][0] for ind in generation_best]

    # Plot 1: Average layers vs best individual layers
    ax1.plot(
        generations,
        avg_layers,
        "b-",
        linewidth=2,
        marker="o",
        label="Population Average",
    )
    ax1.plot(
        generations,
        best_layers,
        "r-",
        linewidth=2,
        marker="s",
        label="Best Individual",
    )
    ax1.set_xlabel("Generation")
    ax1.set_ylabel("Number of Layers")
    ax1.set_title("Layer Count Evolution")
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Plot 2: Layer distribution in final population
    final_pop = logbook[-1]["population"]
    final_layers = [ind[0] for ind in final_pop]

    unique_layers = sorted(set(final_layers))
    counts = [final_layers.count(l) for l in unique_layers]

    ax2.bar(unique_layers, counts, color="purple", alpha=0.7)
    ax2.set_xlabel("Number of Layers")
    ax2.set_ylabel("Count")
    ax2.set_title("Final Population Layer Distribution")
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig("plots/layers_evolution.png", dpi=300, bbox_inches="tight")
    plt.show()


def create_comprehensive_summary_plot(logbook, generation_best) -> None:
    """Create a comprehensive summary plot with key metrics."""
    fig = plt.figure(figsize=(20, 16))
    gs = fig.add_gridspec(4, 3, hspace=0.3, wspace=0.3)

    # Extract all data
    generations = [record["gen"] for record in logbook]
    max_fitness = [record["max"] for record in logbook]
    avg_fitness = [record["avg"] for record in logbook]

    gen_data = []
    fitness_data = []
    beta_data = []
    lr_data = []
    layers_data = []
    param_counts = []

    for gen, individual, fitness in generation_best:
        gen_data.append(gen)
        fitness_data.append(fitness)
        num_layers, layer_sizes, beta, lr, _dropout = parse_individual(
            individual,
        )
        beta_data.append(beta)
        lr_data.append(lr)
        layers_data.append(num_layers)

        # Calculate parameters
        params = 784 * layer_sizes[0]
        for i in range(len(layer_sizes) - 1):
            params += layer_sizes[i] * layer_sizes[i + 1]
        params += layer_sizes[-1] * 10
        param_counts.append(params)

    # Main fitness plot (spans 2 columns)
    ax_main = fig.add_subplot(gs[0, :2])
    ax_main.plot(
        generations,
        max_fitness,
        "r-",
        linewidth=3,
        label="Best Individual",
        marker="o",
        markersize=6,
    )
    ax_main.plot(
        generations,
        avg_fitness,
        "b-",
        linewidth=2,
        label="Population Avg",
        alpha=0.7,
    )
    ax_main.plot(
        generations,
        backprop_accuracies[:len(generations)],
        "g-",
        linewidth=3,
        label="PyTorch",
        marker="*",
        markersize=10,
    )
    ax_main.set_xlabel("Generation", fontsize=12)
    ax_main.set_ylabel("Fitness", fontsize=12)
    ax_main.set_title(
        "Evolution Summary - Fitness Progression",
        fontsize=14,
        fontweight="bold",
    )
    ax_main.legend(fontsize=11)
    ax_main.grid(True, alpha=0.3)

    # Best individual info (right column, top)
    ax_info = fig.add_subplot(gs[0, 2])
    ax_info.axis("off")
    best_idx = fitness_data.index(max(fitness_data))
    best_gen = gen_data[best_idx]
    best_fitness = fitness_data[best_idx]
    best_individual = generation_best[best_idx][1]

    _, best_layers, best_beta, best_lr, best_dropout = parse_individual(
        best_individual,
    )

    info_text = f"""BEST INDIVIDUAL

Generation: {best_gen}
Fitness: {best_fitness:.6f}

Architecture:
• Layers: {len(best_layers)}
• Sizes: {best_layers}
• Parameters: ~{param_counts[best_idx]:,}

Parameters:
• Beta: {best_beta:.4f}
• Learning Rate: {best_lr:.6f}
• Dropout: {best_dropout:.4f}"""

    ax_info.text(
        0.05,
        0.95,
        info_text,
        transform=ax_info.transAxes,
        fontsize=10,
        verticalalignment="top",
        bbox={
            "boxstyle": "round,pad=0.5",
            "facecolor": "lightblue",
            "alpha": 0.8,
        },
    )

    # Architecture depth evolution
    ax_depth = fig.add_subplot(gs[1, 0])
    ax_depth.plot(
        gen_data,
        layers_data,
        "purple",
        linewidth=2,
        marker="s",
        markersize=6,
    )
    ax_depth.set_xlabel("Generation")
    ax_depth.set_ylabel("Number of Layers")
    ax_depth.set_title("Architecture Depth")
    ax_depth.grid(True, alpha=0.3)
    ax_depth.set_ylim(1.5, 4.5)

    # Parameter count evolution
    ax_params = fig.add_subplot(gs[1, 1])
    ax_params.plot(
        gen_data,
        param_counts,
        "orange",
        linewidth=2,
        marker="^",
        markersize=6,
    )
    ax_params.set_xlabel("Generation")
    ax_params.set_ylabel("Parameters")
    ax_params.set_title("Model Complexity")
    ax_params.grid(True, alpha=0.3)
    ax_params.yaxis.set_major_formatter(
        plt.FuncFormatter(lambda x, p: f"{x / 1000:.0f}K"),
    )

    # Beta evolution
    ax_beta = fig.add_subplot(gs[1, 2])
    ax_beta.plot(
        gen_data,
        beta_data,
        "red",
        linewidth=2,
        marker="o",
        markersize=6,
    )
    ax_beta.set_xlabel("Generation")
    ax_beta.set_ylabel("Beta Value")
    ax_beta.set_title("Beta Parameter")
    ax_beta.grid(True, alpha=0.3)
    ax_beta.set_ylim(0.65, 1.0)

    # Performance vs complexity scatter
    ax_scatter = fig.add_subplot(gs[2, 0])
    scatter = ax_scatter.scatter(
        param_counts,
        fitness_data,
        c=gen_data,
        cmap="viridis",
        s=100,
        alpha=0.7,
        edgecolors="black",
    )
    ax_scatter.set_xlabel("Parameter Count")
    ax_scatter.set_ylabel("Fitness")
    ax_scatter.set_title("Performance vs Complexity")
    ax_scatter.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax_scatter, label="Generation")

    # Learning rate evolution
    ax_lr = fig.add_subplot(gs[2, 1])
    ax_lr.plot(
        gen_data,
        lr_data,
        "green",
        linewidth=2,
        marker="d",
        markersize=6,
    )
    ax_lr.set_xlabel("Generation")
    ax_lr.set_ylabel("Learning Rate")
    ax_lr.set_title("Learning Rate Evolution")
    ax_lr.grid(True, alpha=0.3)
    ax_lr.ticklabel_format(style="scientific", axis="y", scilimits=(0, 0))

    # Improvement rate
    ax_improve = fig.add_subplot(gs[2, 2])
    improvement_rate = [0]  # First generation has no improvement
    improvement_rate.extend(
        fitness_data[i] - fitness_data[i - 1]
        for i in range(1, len(fitness_data))
    )

    colors = ["red" if x < 0 else "green" for x in improvement_rate[1:]]
    ax_improve.bar(gen_data[1:], improvement_rate[1:], color=colors, alpha=0.7)
    ax_improve.axhline(y=0, color="black", linestyle="-", alpha=0.3)
    ax_improve.set_xlabel("Generation")
    ax_improve.set_ylabel("Fitness Improvement")
    ax_improve.set_title("Generation-to-Generation Progress")
    ax_improve.grid(True, alpha=0.3)

    # Final statistics summary
    ax_stats = fig.add_subplot(gs[3, :])
    ax_stats.axis("off")

    # Calculate statistics
    total_improvement = (
        fitness_data[-1] - fitness_data[0] if len(fitness_data) > 1 else 0
    )
    avg_improvement = (
        total_improvement / len(fitness_data) if len(fitness_data) > 1 else 0
    )
    best_improvement_gen = (
        improvement_rate.index(max(improvement_rate[1:])) + 1
        if len(improvement_rate) > 1
        else 0
    )

    convergence_gen = len(fitness_data)
    for i in range(len(fitness_data) - 5, 0, -1):
        if abs(fitness_data[i] - fitness_data[-1]) > 0.001:
            convergence_gen = i + 5
            break

    stats_text = f"""EVOLUTION STATISTICS

Total Generations: {len(fitness_data)}                    Total Improvement: {total_improvement:.6f}                    Average per Generation: {avg_improvement:.6f}

Best Generation: {best_gen}                    Convergence: ~Gen {convergence_gen}                    Best Improvement: Gen {best_improvement_gen} (+{max(improvement_rate[1:]) if len(improvement_rate) > 1 else 0:.6f})

Architecture Range: {min(layers_data)}-{max(layers_data)} layers          Parameter Range: {min(param_counts):,}-{max(param_counts):,}          Beta Range: {min(beta_data):.3f}-{max(beta_data):.3f}"""

    ax_stats.text(
        0.5,
        0.5,
        stats_text,
        transform=ax_stats.transAxes,
        fontsize=12,
        horizontalalignment="center",
        verticalalignment="center",
        bbox={
            "boxstyle": "round,pad=1",
            "facecolor": "lightyellow",
            "alpha": 0.8,
        },
    )

    plt.suptitle(
        "SNN Evolution Comprehensive Analysis",
        fontsize=16,
        fontweight="bold",
        y=0.98,
    )
    plt.savefig(
        "plots/comprehensive_evolution_summary.png",
        dpi=300,
        bbox_inches="tight",
    )
    plt.show()


Set up  DEAP

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

def get_num_layers(individual):
    return len(individual[0])

def create_individual():
    """Create an individual and wrap it in creator.Individual."""
    ind_data = create_individual_with_depth()
    return creator.Individual(ind_data)


toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate_model_with_depth)
toolbox.register("mate", crossover_with_depth)
toolbox.register(
    "mutate",
    bounded_mutation_with_depth,
    mu=0,
    sigma=0.1,
    indpb=0.15,
)
toolbox.register("select", tools.selTournament, tournsize=POP_SIZE // 4)


Running the evolution

In [ ]:
def run_evolution():
    pop_size = POP_SIZE
    pop = toolbox.population(POP_SIZE)
    hof = tools.HallOfFame(POP_SIZE // 4)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("max", np.max)
    stats.register("min", np.min)
    stats.register("std", np.std)

    # Add a new statistic for average layers
    stats_layers = tools.Statistics(
        operator.itemgetter(0),
    )  # ind[0] is the number of layers
    stats_layers.register("avg", np.mean)

    logbook = tools.Logbook()
    logbook.header = ["gen", "nevals", *stats.fields]

    # Store best individuals from each generation
    generation_best = []
    # Store populations for each generation
    populations = []
    # Store average layers per generation
    avg_layers_per_gen = []

    console.print("Starting Evolution...")
    console.print("=" * 80)

    # Initial evaluation
    fitnesses = toolbox.map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses, strict=False):
        ind.fitness.values = fit

    hof.update(pop)
    populations.append(pop)  # Store initial population

    # Calculate and store average layers for generation 0
    avg_layers = stats_layers.compile(pop)["avg"]
    avg_layers_per_gen.append(avg_layers)

    # Find and display best individual of generation 0
    best_ind = tools.selBest(pop, 1)[0]
    generation_best.append((0, best_ind.copy(), best_ind.fitness.values[0]))
    print_best_individual(0, best_ind, best_ind.fitness.values[0])

    record = stats.compile(pop)
    logbook.record(gen=0, nevals=len(pop), **record)
    console.print(f"\nGen 0 Stats:\n {logbook.stream}")
    console.print(f"Average layers in population: {avg_layers:.2f}")

    # Evolution loop
    for gen in range(1, 31):
        console.print(f"\n--- Processing Generation {gen} ---")

        # Selection and reproduction
        offspring = tools.selTournament(
            individuals=pop,
            k=POP_SIZE // 2,
            tournsize=POP_SIZE // 4,
        )
        offspring = list(map(toolbox.clone, offspring))

        # Crossover
        for child1, child2 in zip(
            offspring[::2],
            offspring[1::2],
            strict=False,
        ):
            if RNG.random() < 0.6:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        # Mutation
        for mutant in offspring:
            toolbox.mutate(mutant)
            del mutant.fitness.values

        # Evaluation
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses, strict=False):
            ind.fitness.values = fit

        # Elitism: Keep best individuals
        pop[:] = tools.selTournament(
            individuals=pop + offspring,
            k=POP_SIZE,
            tournsize=POP_SIZE // 4,
        )

        hof.update(pop)
        populations.append(pop.copy())  # Store current population

        # Calculate and store average layers for this generation
        avg_layers = stats_layers.compile(pop)["avg"]
        avg_layers_per_gen.append(avg_layers)

        # Find and display best individual of current generation
        best_ind = tools.selBest(pop, 1)[0]
        generation_best.append((
            gen,
            best_ind.copy(),
            best_ind.fitness.values[0],
        ))
        print_best_individual(gen, best_ind, best_ind.fitness.values[0])

        record = stats.compile(pop)
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        console.print(f"Gen {gen} \nStats: {logbook.stream}")
        console.print(f"Average layers in population: {avg_layers:.2f}")

        # Early stopping if no improvement
        if gen > 10 and logbook[-1]["max"] == logbook[-5]["max"]:
            console.print(
                f"\nEarly stopping at generation {gen} due to no improvement",
            )
            break

    # Add population information to logbook
    for i, entry in enumerate(logbook):
        entry["population"] = populations[i]
        entry["avg_layers"] = avg_layers_per_gen[
            i
        ]  # Add average layers to logbook

    # Plot the average layers evolution
    plot_layers_evolution(logbook, generation_best)

    # Final summary
    console.print("\n" + "=" * 80)
    console.print("EVOLUTION COMPLETE - FINAL SUMMARY")
    console.print("=" * 80)

    # Print overall best
    best_overall = max(generation_best, key=operator.itemgetter(2))
    best_gen, best_ind, best_fitness = best_overall

    console.print(f"\nOVERALL BEST INDIVIDUAL (from Generation {best_gen}):")
    print_best_individual("FINAL", best_ind, best_fitness)

    # Print evolution summary
    console.print("\nEvolution Summary:")
    console.print(f"- Total Generations: {gen}")
    console.print(f"- Population Size: {pop_size}")
    console.print(f"- Best Fitness Achieved: {best_fitness:.6f}")
    console.print(f"- Best Found in Generation: {best_gen}")

    # Generate all plots
    console.print("\nGenerating plots...")

    console.print("1. Evolution Progress Plot...")
    plot_evolution_progress(logbook, generation_best)

    console.print("2. Architecture Evolution Plot...")
    plot_architecture_evolution(generation_best)

    console.print("3. Beta Evolution Plot...")
    plot_beta_evolution(generation_best)

    console.print("4. Comprehensive Summary Plot...")
    create_comprehensive_summary_plot(logbook, generation_best)

    console.print("All plots saved successfully!")

    return pop, logbook, hof, generation_best


if __name__ == "__main__":
    population, logbook, hall_of_fame, best_individuals = run_evolution()

Starting Evolution...

GENERATION 0 - BEST INDIVIDUAL
Fitness: 0.819501
Architecture: 784 → 237 → 170 → 10
Beta (β): 0.7480
Learning Rate: 0.003974
Dropout Rate: 0.0105
Total Parameters: ~500,817

Gen 0 Stats:
 gen	nevals	avg     	max     	min      	std     
0  	400   	0.491303	0.819501	0.0690844	0.224451

--- Processing Generation 1 ---

GENERATION 1 - BEST INDIVIDUAL
Fitness: 0.827071
Architecture: 784 → 244 → 160 → 10
Beta (β): 0.7997
Learning Rate: 0.003889
Dropout Rate: 0.1736
Total Parameters: ~495,972
Gen 1 
Stats: 1  	266   	0.725701	0.827071	0.385623 	0.0836032

--- Processing Generation 2 ---

GENERATION 2 - BEST INDIVIDUAL
Fitness: 0.828989
Architecture: 784 → 252 → 173 → 10
Beta (β): 0.9322
Learning Rate: 0.004020
Dropout Rate: 0.0393
Total Parameters: ~530,265
Gen 2 
Stats: 2  	276   	0.777489	0.828989	0.706    	0.0257494

--- Processing Generation 3 ---

GENERATION 3 - BEST INDIVIDUAL
Fitness: 0.831121
Architecture: 784 → 217 → 149 → 10
Beta (β): 0.8325
Learning Rate: 0.0